In [1]:
import cv2
import torch
import numpy as np


In [2]:
import torch.nn as nn
import torch.nn.functional as F
from utils.tensorboard import TensorBoard
from Renderer.model import FCN
from Renderer.stroke_gen import *

In [3]:
writer = TensorBoard("../train_log/")
import torch.optim as optim

criterion = nn.MSELoss()
net = FCN()
optimizer = optim.Adam(net.parameters(), lr=3e-6)
batch_size = 64

use_cuda = torch.cuda.is_available()
step = 0

In [4]:
def save_model():
    if use_cuda:
        net.cpu()
    torch.save(net.state_dict(), "./model/renderer_oil.pkl")
    if use_cuda:
        net.cuda()


def load_weights():
    pretrained_dict = torch.load("./model/renderer_oil.pkl")
    model_dict = net.state_dict()
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    model_dict.update(pretrained_dict)
    net.load_state_dict(model_dict)


# load_weights()
# 500000
while step < 50000:
    net.train()
    train_batch = []
    ground_truth = []
    for i in range(batch_size):
        f = np.random.uniform(0, 1, 10)
        train_batch.append(f)
        ground_truth.append(draw_oil(f))

    train_batch = torch.tensor(train_batch).float()
    ground_truth = torch.tensor(ground_truth).float()
    if use_cuda:
        net = net.cuda()
        train_batch = train_batch.cuda()
        ground_truth = ground_truth.cuda()
    gen = net(train_batch)
    optimizer.zero_grad()
    loss = criterion(gen, ground_truth)
    loss.backward()
    optimizer.step()
    print(step, loss.item())
#     if step < 200000:
#         lr = 1e-4
#     elif step < 400000:
#         lr = 1e-5
#     else:
#         lr = 1e-6
    if step < 20000:
        lr = 1e-4
    elif step < 40000:
        lr = 1e-5
    else:
        lr = 1e-6
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr
    writer.add_scalar("train/loss", loss.item(), step)
    if step % 10 == 0:
        net.eval()
        gen = net(train_batch)
        loss = criterion(gen, ground_truth)
        writer.add_scalar("val/loss", loss.item(), step)
        for i in range(32):
            G = gen[i].cpu().data.numpy()
            GT = ground_truth[i].cpu().data.numpy()
            writer.add_image("train/gen{}.png".format(i), G, step)
            writer.add_image("train/ground_truth{}.png".format(i), GT, step)
    if step % 100 == 0:
        save_model()
    step += 1

0 0.24148191511631012
1 0.23806609213352203
2 0.23908543586730957
3 0.23993030190467834
4 0.23876875638961792
5 0.23940086364746094
6 0.23811882734298706
7 0.23710742592811584
8 0.24110610783100128
9 0.23716336488723755
10 0.23838971555233002
11 0.23913200199604034
12 0.23858687281608582
13 0.23759883642196655
14 0.23865269124507904
15 0.23540861904621124
16 0.2400895208120346
17 0.2385418713092804
18 0.23969560861587524
19 0.24088400602340698
20 0.23783373832702637
21 0.24293583631515503
22 0.23721100389957428
23 0.23647084832191467
24 0.23998627066612244
25 0.238552525639534
26 0.23872587084770203
27 0.23834620416164398
28 0.23900176584720612
29 0.23848283290863037
30 0.2373669594526291
31 0.23844866454601288
32 0.23697395622730255
33 0.23946866393089294
34 0.23907920718193054
35 0.23804496228694916
36 0.2384909838438034
37 0.2379031479358673
38 0.2379358857870102
39 0.23802657425403595
40 0.23900257050991058
41 0.23814697563648224
42 0.2407148778438568
43 0.2381444275379181
44 0.236

351 0.07820557057857513
352 0.08207181096076965
353 0.07430862635374069
354 0.07516919076442719
355 0.07427681982517242
356 0.06120743602514267
357 0.069159597158432
358 0.0810026228427887
359 0.07153749465942383
360 0.06887764483690262
361 0.072712741792202
362 0.06396610289812088
363 0.07954881340265274
364 0.07470903545618057
365 0.07619305700063705
366 0.07263888418674469
367 0.07017064094543457
368 0.08220414072275162
369 0.06739411503076553
370 0.06692498177289963
371 0.08098075538873672
372 0.06535691022872925
373 0.07188410311937332
374 0.0796561911702156
375 0.07801167666912079
376 0.07447266578674316
377 0.07305232435464859
378 0.07633502036333084
379 0.07090374082326889
380 0.06871484965085983
381 0.0750785544514656
382 0.0787520557641983
383 0.06969485431909561
384 0.062172092497348785
385 0.07885543256998062
386 0.06707494705915451
387 0.07500439137220383
388 0.07285847514867783
389 0.0827263593673706
390 0.07149260491132736
391 0.06845131516456604
392 0.06677297502756119


694 0.05581030249595642
695 0.06476530432701111
696 0.05570746585726738
697 0.059722110629081726
698 0.06876740604639053
699 0.0546235665678978
700 0.05849682167172432
701 0.06113428622484207
702 0.05642560124397278
703 0.06419868022203445
704 0.06030489131808281
705 0.05768934637308121
706 0.06467580050230026
707 0.058924004435539246
708 0.05444086715579033
709 0.06828237324953079
710 0.06172154098749161
711 0.05836474150419235
712 0.06112970411777496
713 0.05811992287635803
714 0.0632956326007843
715 0.06876585632562637
716 0.06195522099733353
717 0.06338028609752655
718 0.054576627910137177
719 0.06454893946647644
720 0.05842491239309311
721 0.06543423980474472
722 0.06626205146312714
723 0.05867811664938927
724 0.056499749422073364
725 0.0662224069237709
726 0.059112388640642166
727 0.05488031357526779
728 0.05538341403007507
729 0.059272877871990204
730 0.055729784071445465
731 0.05778449773788452
732 0.05902322009205818
733 0.06191159039735794
734 0.05842489004135132
735 0.059286

1032 0.04692374914884567
1033 0.04238389804959297
1034 0.04573540762066841
1035 0.046287745237350464
1036 0.046018052846193314
1037 0.04339038208127022
1038 0.045191630721092224
1039 0.04274711757898331
1040 0.045216646045446396
1041 0.04390186071395874
1042 0.042905718088150024
1043 0.04585575312376022
1044 0.04032840579748154
1045 0.04377488046884537
1046 0.04214652627706528
1047 0.044121306389570236
1048 0.04941801726818085
1049 0.041810281574726105
1050 0.04371432960033417
1051 0.04656311497092247
1052 0.04375629127025604
1053 0.04138529300689697
1054 0.0411003902554512
1055 0.04199850186705589
1056 0.04330053925514221
1057 0.042426250874996185
1058 0.04894988611340523
1059 0.04324088245630264
1060 0.04259107634425163
1061 0.037394993007183075
1062 0.04528550058603287
1063 0.04454349726438522
1064 0.044474001973867416
1065 0.03997499495744705
1066 0.04711833596229553
1067 0.04145646467804909
1068 0.039937637746334076
1069 0.04586215317249298
1070 0.041429318487644196
1071 0.0430329

1358 0.041572947055101395
1359 0.03745461255311966
1360 0.037132661789655685
1361 0.03857063874602318
1362 0.03746671974658966
1363 0.04264558106660843
1364 0.03949664905667305
1365 0.03859204426407814
1366 0.04148344323039055
1367 0.0378769114613533
1368 0.037950173020362854
1369 0.033902041614055634
1370 0.037276361137628555
1371 0.03986059129238129
1372 0.03728017583489418
1373 0.037886131554841995
1374 0.03877008706331253
1375 0.039615415036678314
1376 0.03641457483172417
1377 0.03896541893482208
1378 0.03532962501049042
1379 0.04086310788989067
1380 0.03582099825143814
1381 0.03496454656124115
1382 0.040610864758491516
1383 0.04261389374732971
1384 0.03925615921616554
1385 0.03910909965634346
1386 0.040346480906009674
1387 0.03862098976969719
1388 0.03633852303028107
1389 0.037556856870651245
1390 0.039377085864543915
1391 0.04161854460835457
1392 0.04013141617178917
1393 0.03857680410146713
1394 0.03606302663683891
1395 0.03889536112546921
1396 0.03456611558794975
1397 0.03699225

...

48589 0.0022413909900933504
48590 0.0022839619778096676
48591 0.002153781009837985
48592 0.002264810958877206
48593 0.00225820392370224
48594 0.002340213628485799
48595 0.0021396863739937544
48596 0.0023854277096688747
48597 0.002281283028423786
48598 0.002243337919935584
48599 0.0022094636224210262
48600 0.002531742677092552
48601 0.002372927498072386
48602 0.0023056555073708296
48603 0.00213715061545372
48604 0.0023820698261260986
48605 0.0022228178568184376
48606 0.0023698550648987293
48607 0.002334002871066332
48608 0.002493994077667594
48609 0.0024509953800588846
48610 0.002200083574280143
48611 0.002435876289382577
48612 0.002446951111778617
48613 0.0024098330177366734
48614 0.0021373280324041843
48615 0.0021369934547692537
48616 0.002104026498273015
48617 0.002231230493634939
48618 0.0025393052492290735
48619 0.0022790769580751657
48620 0.002231688005849719
48621 0.002455660840496421
48622 0.0024942492600530386
48623 0.002265708055347204
48624 0.001956869848072529
48625 0.002440

48887 0.0022827403154224157
48888 0.0026447332929819822
48889 0.0022254916839301586
48890 0.002273437799885869
48891 0.0027306729461997747
48892 0.0022639345843344927
48893 0.002245693700388074
48894 0.0022938610054552555
48895 0.0021232415456324816
48896 0.0024594366550445557
48897 0.002181215677410364
48898 0.0022872216068208218
48899 0.0024347782600671053
48900 0.002267620526254177
48901 0.0023512965999543667
48902 0.0024051957298070192
48903 0.0022077036555856466
48904 0.0023225455079227686
48905 0.002206644741818309
48906 0.002201089635491371
48907 0.0022259659599512815
48908 0.002131660468876362
48909 0.0020484065171331167
48910 0.002450803527608514
48911 0.0024536934215575457
48912 0.002200692193582654
48913 0.0023259182926267385
48914 0.002559095621109009
48915 0.002406938700005412
48916 0.002335774479433894
48917 0.002463857177644968
48918 0.002253968268632889
48919 0.0020817555487155914
48920 0.002133076312020421
48921 0.002089882967993617
48922 0.0022553179878741503
48923 0.

49187 0.0023275092244148254
49188 0.0024562065955251455
49189 0.0024818237870931625
49190 0.0022119178902357817
49191 0.0022222138941287994
49192 0.002333829877898097
49193 0.002214521635323763
49194 0.002189480932429433
49195 0.0022490595001727343
49196 0.002543446607887745
49197 0.0022796958219259977
49198 0.0020636143162846565
49199 0.002357585821300745
49200 0.002353107323870063
49201 0.0021270615980029106
49202 0.0021758091170340776
49203 0.0022711323108524084
49204 0.0023565248120576143
49205 0.0023422990925610065
49206 0.0022573985625058413
49207 0.0022633569315075874
49208 0.002156204776838422
49209 0.0022413176484405994
49210 0.002392818219959736
49211 0.0022876679431647062
49212 0.002279038541018963
49213 0.0022025960497558117
49214 0.0022012614645063877
49215 0.0024132144171744585
49216 0.002268511336296797
49217 0.0022284446749836206
49218 0.0024013989605009556
49219 0.0022060866467654705
49220 0.002168521285057068
49221 0.0023156660608947277
49222 0.002549987519159913
4922

49485 0.0022133418824523687
49486 0.0025023575872182846
49487 0.002214087639003992
49488 0.0021281461231410503
49489 0.002132838126271963
49490 0.0022440438624471426
49491 0.0024206165689975023
49492 0.0022103830706328154
49493 0.002196026034653187
49494 0.0021498240530490875
49495 0.0023664780892431736
49496 0.0020227795466780663
49497 0.0022650265600532293
49498 0.0021558990702033043
49499 0.0023730432149022818
49500 0.0023112620692700148
49501 0.002370135858654976
49502 0.002428179606795311
49503 0.0023556621745228767
49504 0.0022393951658159494
49505 0.0021746275015175343
49506 0.00223241513594985
49507 0.002244082512333989
49508 0.002452180255204439
49509 0.0021622132044285536
49510 0.002434787340462208
49511 0.0021522215101867914
49512 0.0023166202008724213
49513 0.002343548694625497
49514 0.0023110066540539265
49515 0.0022544183302670717
49516 0.002217421308159828
49517 0.002147737890481949
49518 0.0024784666020423174
49519 0.0023110397160053253
49520 0.002205644967034459
49521 

49785 0.0022814932744950056
49786 0.0023685013875365257
49787 0.0021768053993582726
49788 0.0021626208908855915
49789 0.0021360486280173063
49790 0.0022224991116672754
49791 0.0023292505647987127
49792 0.0024725901894271374
49793 0.0022376924753189087
49794 0.0023484239354729652
49795 0.002126590348780155
49796 0.0022109101992100477
49797 0.002218194305896759
49798 0.0023467540740966797
49799 0.0023888677824288607
49800 0.002194356871768832
49801 0.0023240316659212112
49802 0.0022440077736973763
49803 0.0023240097798407078
49804 0.00223794998601079
49805 0.00242041377350688
49806 0.002311001531779766
49807 0.0022831412497907877
49808 0.0023392890579998493
49809 0.002122151432558894
49810 0.0022734152153134346
49811 0.002212405437603593
49812 0.002204220276325941
49813 0.0024067824706435204
49814 0.002542059402912855
49815 0.0023477417416870594
49816 0.002403992461040616
49817 0.002584712579846382
49818 0.002527008531615138
49819 0.002004378940910101
49820 0.0021338392980396748
49821 0.